In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('Family data - Transactions.csv')

In [5]:
df

,Date,Category,Sub-Category,Amount_DR,Amount_CR,Entity,Notes,Payment_Mode,Cycle_ID
0,2026-01-04,Personal Spending,Utilities,666.0,0.00,Mobile,Recharge (For me),UPI,Dec-2025
1,2026-01-04,Personal Spending,Food & Dining,30.0,0.00,Local Shop,Masal Puri,UPI,Dec-2025
2,2026-01-03,Personal Spending,Food & Dining,168.0,0.00,Local Shop,Curri gravy,UPI,Dec-2025
3,2026-01-03,Share Investment,Refund,0.0,269.26,INDMoney,Unused Amount,UPI,Dec-2025
4,2026-01-02,Savings Transfer,Savings,0.0,562.20,IPPB,IPPB,UPI,Dec-2025
...,...,...,...,...,...,...,...,...,...
169,2025-09-25,Family Support,P2P Inflow,0.0,1000.00,Dad,From Dad,UPI,Sep-2025
170,2025-09-18,Personal Spending,Transport,15.0,0.00,Metro,Metro,UPI,Sep-2025
171,2025-09-04,Personal Spending,Misc,2.0,0.00,G-Play,G-Play,UPI,Sep-2025
172,2025-09-04,Personal Spending,Misc,0.0,2.00,G-Play,G-Play,UPI,Sep-2025


In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
df['Month_Year_Sort'] = df['Date'].dt.to_period('M')
df['Month_Year'] = df['Date'].dt.strftime('%b-%Y')

In [8]:
df.fillna(0, inplace=True)

In [9]:
def aggregate_monthly(x):
    return pd.Series({
        # Income Logic: Assumes all CR is Salary unless specified otherwise
        # You can adjust this logic if you have a 'Other Income' category
        'Total_Salary_Received': x['Amount_CR'].sum(),
        'Total_Other_Income': 0, 
        
        # Expense Logic: Filters Amount_DR based on Category names
        'Total_Spent_Family': x.loc[x['Category'] == 'Family', 'Amount_DR'].sum(),
        'Total_Spent_Shares': x.loc[x['Category'] == 'Shares', 'Amount_DR'].sum(),
        'Total_Spent_Savings': x.loc[x['Category'] == 'Savings', 'Amount_DR'].sum(),
        'Total_Spent_Personal': x.loc[x['Category'] == 'Personal Spending', 'Amount_DR'].sum()
    })

In [10]:
monthly_df = df.groupby('Month_Year_Sort').apply(aggregate_monthly).reset_index()

C:\Users\Windows\AppData\Local\Temp\ipykernel_10960\1835090437.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_df = df.groupby('Month_Year_Sort').apply(aggregate_monthly).reset_index()


In [11]:
initial_opening_balance = 0 # Change this if you had money before Sep 2025
current_balance = initial_opening_balance

In [12]:
opening_balances = []
closing_balances = []

for index, row in monthly_df.iterrows():
    # Opening Balance for this month is the previous month's closing (or initial)
    opening = current_balance
    
    # Calculate Total Income and Total Expense for this row
    total_income = row['Total_Salary_Received'] + row['Total_Other_Income']
    total_expense = (row['Total_Spent_Family'] + 
                     row['Total_Spent_Shares'] + 
                     row['Total_Spent_Savings'] + 
                     row['Total_Spent_Personal'])
    
    # Closing Balance formula
    closing = opening + total_income - total_expense
    
    # Append to lists
    opening_balances.append(opening)
    closing_balances.append(closing)
    
    # Update current_balance for next iteration
    current_balance = closing


In [15]:
# Assign the lists back to the DataFrame
monthly_df['Opening_Balance'] = opening_balances
monthly_df['Closing_Balance'] = closing_balances

# Convert the period sort column back to the string format requested
monthly_df['Month_Year'] = monthly_df['Month_Year_Sort'].dt.strftime('%b-%Y')

# Reorder columns to match your exact request
final_cols = [
    'Month_Year', 
    'Opening_Balance', 
    'Total_Salary_Received', 
    'Total_Other_Income', 
    'Total_Spent_Family', 
    'Total_Spent_Shares', 
    'Total_Spent_Savings', 
    'Total_Spent_Personal', 
    'Closing_Balance'
]

final_df = monthly_df[final_cols]

# 7. Display the Output
# Styling for better readability (Optional)
final_df.style.format("{:.2f}", subset=final_df.columns[1:])

,Month_Year,Opening_Balance,Total_Salary_Received,Total_Other_Income,Total_Spent_Family,Total_Spent_Shares,Total_Spent_Savings,Total_Spent_Personal,Closing_Balance
0,Sep-2025,0.00,1002.00,0.00,0.00,0.00,0.00,17.00,985.00
1,Oct-2025,985.00,750.00,0.00,0.00,0.00,0.00,910.00,825.00
2,Nov-2025,825.00,10580.00,0.00,0.00,0.00,0.00,1994.50,9410.50
3,Dec-2025,9410.50,20982.10,0.00,0.00,0.00,0.00,6326.94,24065.66
4,Jan-2026,24065.66,831.46,0.00,0.00,0.00,0.00,936.00,23961.12


In [16]:
%pip install jinja2

Note: you may need to restart the kernel to use updated packages.


In [17]:
# 1. Load the Data
df = pd.read_csv('Family data - Transactions.csv')

# 2. Preprocessing
df['Date'] = pd.to_datetime(df['Date'])
df['Month_Year_Sort'] = df['Date'].dt.to_period('M')

# Fill NaN values
df.fillna(0, inplace=True)

# 3. Define aggregation function (CORRECTED CATEGORY NAMES)
def aggregate_monthly(x):
    return pd.Series({
        # Income Logic: Sums all Credit (CR) amounts
        'Total_Salary_Received': x['Amount_CR'].sum(),
        'Total_Other_Income': 0, 
        
        # Expense Logic: Updated to match your actual CSV data
        'Total_Spent_Family': x.loc[x['Category'] == 'Family Support', 'Amount_DR'].sum(),
        'Total_Spent_Shares': x.loc[x['Category'] == 'Share Investment', 'Amount_DR'].sum(),
        'Total_Spent_Savings': x.loc[x['Category'] == 'Savings Transfer', 'Amount_DR'].sum(),
        'Total_Spent_Personal': x.loc[x['Category'] == 'Personal Spending', 'Amount_DR'].sum()
    })

# 4. Group by Month 
monthly_df = df.groupby('Month_Year_Sort').apply(aggregate_monthly, include_groups=False).reset_index()

# 5. Calculate Balances (Cumulative Logic)
# If you have money starting from before Sep 2025, change this value.
initial_opening_balance = 0 
current_balance = initial_opening_balance

opening_balances = []
closing_balances = []

for index, row in monthly_df.iterrows():
    opening = current_balance
    
    # Total Income = Salary + Other
    total_income = row['Total_Salary_Received'] + row['Total_Other_Income']
    
    # Total Expenses = Sum of all expense columns
    total_expense = (row['Total_Spent_Family'] + 
                     row['Total_Spent_Shares'] + 
                     row['Total_Spent_Savings'] + 
                     row['Total_Spent_Personal'])
    
    # Balance Formula
    closing = opening + total_income - total_expense
    
    opening_balances.append(opening)
    closing_balances.append(closing)
    
    current_balance = closing

monthly_df['Opening_Balance'] = opening_balances
monthly_df['Closing_Balance'] = closing_balances

# 6. Final Formatting
monthly_df['Month_Year'] = monthly_df['Month_Year_Sort'].dt.strftime('%b-%Y')

final_cols = [
    'Month_Year', 
    'Opening_Balance', 
    'Total_Salary_Received', 
    'Total_Other_Income', 
    'Total_Spent_Family', 
    'Total_Spent_Shares', 
    'Total_Spent_Savings', 
    'Total_Spent_Personal', 
    'Closing_Balance'
]

final_df = monthly_df[final_cols]

# 7. Display Output
final_df

,Month_Year,Opening_Balance,Total_Salary_Received,Total_Other_Income,Total_Spent_Family,Total_Spent_Shares,Total_Spent_Savings,Total_Spent_Personal,Closing_Balance
0,Sep-2025,0.00,1002.00,0.0,0.0,20.0,0.0,17.00,965.00
1,Oct-2025,965.00,750.00,0.0,0.0,0.0,0.0,910.00,805.00
2,Nov-2025,805.00,10580.00,0.0,0.0,0.0,1000.0,1994.50,8390.50
3,Dec-2025,8390.50,20982.10,0.0,0.0,1460.8,0.0,6326.94,21584.86
4,Jan-2026,21584.86,831.46,0.0,0.0,0.0,0.0,936.00,21480.32
